In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf 

In [ ]:
def download(ticker, start, end):
    return yf.download(ticker, start=start, end=end)

In [ ]:
data = download(ticker=TICKER, start=START, end=END)

In [ ]:

TICKER = "IBM"
START = "2020-01-01"
END = "2025-01-01"
PERIOD = 20
COEF = 2
"""BBAND visualization"""
df = data.copy()
df["tp"] = (df["Close"] + df["Low"]+df["High"])/3
df["STD"] = df["tp"].rolling(PERIOD).std()
df["MA"] = df["tp"].rolling(PERIOD).mean()
df["Upper"] = df["MA"] + COEF*df["STD"]
df["Lower"] = df["MA"] - COEF*df["STD"]
df.dropna(inplace=True)

def plot(data):
    plt.plot(data["Close"])
    plt.plot(data["Upper"], color="orange")
    plt.plot(data["Lower"], color="orange")
    plt.fill_between(data.index, data["Upper"],
                     data["Lower"], facecolor="orange", alpha=0.1)
    
    plt.title("BBANDs")
    plt.show()
    
plot(data=df)

In [ ]:
import backtrader as bt
from datetime import datetime

In [ ]:
"""Strategy"""
class BBANDStrategy(bt.Strategy):
    params = (
        ("period", 20),
        ("std", 2),
        ("size", 20),
    )

    def __init__(self):
        self.bollinger = bt.indicators.BollingerBands(
            self.data.close, period=self.p.period, devfactor=self.p.std
        )

    def next(self):
        if not self.position:
            if self.data.close[0] > self.bollinger.lines.top[0]:
                self.sell(size=self.p.size)
            elif self.data.close[0] < self.bollinger.lines.bot[0]:
                self.buy(size=self.p.size)
        else:
            if self.position.size > 0:
                self.sell(exectype=bt.Order.Limit, price=self.bollinger.lines.mid[0], size=self.p.size)
            else:
                self.buy(exectype=bt.Order.Limit, price=self.bollinger.lines.mid[0], size=self.p.size)


: 

In [ ]:
cerebro = bt.Cerebro()
cerebro.addstrategy(BBANDStrategy)
df = yf.Ticker("IBM").history(start="2010-01-01", end="2020-01-01")
print(df)

data = bt.feeds.PandasData(dataname=df)
cerebro.adddata(data)
cerebro.addobserver(bt.observers.Value)
cerebro.addanalyzer(bt.analyzers.SharpeRatio, riskfreerate=0.0)
cerebro.addanalyzer(bt.analyzers.Returns)
cerebro.addanalyzer(bt.analyzers.DrawDown)



print(f"Capital: {cerebro.broker.getvalue()}")


In [ ]:
results = cerebro.run()
print(f"Sharpe: {results[0].analyzers.sharperatio.get_analysis()['sharperatio']}")
print(f"Ann. Return: {results[0].analyzers.returns.get_analysis()['rnorm100']}")
print(f"Max Drawdown: {results[0].analyzers.drawdown.get_analysis()['max']['drawdown']}")
print(f"Capital: {cerebro.broker.getvalue()}")
